# TODO

- [ ] Fix the arrays in the data ingestion
- [ ] Fix the datatypes in general
- [X] Fix the connection pooling issue (ClientError: failed to obtain a connection from the pool within 60.0s (timeout))

# Libraries and contants

In [1]:
# !pip install tqdm
# !pip install spacy
# !pip install git+https://github.com/LIAAD/yake

In [2]:
import os
import json
import random

import neo4j
import yake
import numpy as np
from tqdm import tqdm

In [3]:
SEMANTIC_PATH = '../semanticscholar_raw_data'

DEFAULT_JOURNAL_NAME = 'Unknown'

SEED = 13

In [4]:
random.seed(SEED)
np.random.seed(SEED)

# Neo4j

In [5]:
DRIVER = neo4j.GraphDatabase.driver(uri="neo4j://localhost")

In [6]:
def execute(query: str):
    """
    Executes a Cypher @query and returns its result.
    """
    result = DRIVER.execute_query(query)
    return result

In [7]:
def delete_graph() -> None:
    """
    Deletes every node and edge of the graph.
    """
    query = """
        MATCH (n)
        DETACH DELETE n;
    """

    execute(query)

In [8]:
delete_graph()

## Papers

Our dataset does not provide the keywords automatically extracted for us.
Therefore, we will be trying to extract them from the abstract using an external library called [Yake](https://liaad.github.io/yake/).

In [9]:
def parse_journal_name(paper) -> str:
    """
    Not every file has a field 'journal' in the json.
    This function treats those edge cases.
    """
    if 'journal' not in paper or not paper['journal']:
        return DEFAULT_JOURNAL_NAME
    else:
        return paper.get('journal', {'name': DEFAULT_JOURNAL_NAME}).get('name', DEFAULT_JOURNAL_NAME).replace("'", '').replace('"', '')

In [10]:
def sanitize_abstract(abstract: str) -> str:
    if abstract:
        return (
            abstract
            .replace('"', "'")
            .replace('\\', '\\\\')
        )
    else:
        return abstract

In [11]:
def create_papers():
    """
    Create the nodes of label `Paper`.
    """
    # This is used to extract the keywords from the abstract.
    kw_extractor = yake.KeywordExtractor(
        lan='en',
        n=3,  # Max n-gram size
        top=5  # Number of keywords
    )
    
    for fname in tqdm(os.listdir(SEMANTIC_PATH)):
        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())

        title = paper['title'].replace('\\', '')
        keywords = kw_extractor.extract_keywords(paper['abstract']) if paper['abstract'] else ''
        keywords = list(map(lambda x: str.lower(x[0]) if x else '', keywords))

        # publication_venue: "{paper['publicationVenue']}",
        # venue: "{paper['venue']}",
        query = f"""
        CREATE (n:Paper {{
            paper_id: "{paper['paperId']}",
            title: "{title}",
            
            year: toInteger({paper['year'] if paper['year'] else -1}),
            fieldsOfStudy: {paper['fieldsOfStudy'] if paper['fieldsOfStudy'] else '[]'},
            publicationDate: date("{paper['publicationDate'] if paper['publicationDate'] else '1970-01-01'}"),
            abstract: "{sanitize_abstract(paper['abstract'])}",
            keywords: {keywords}
        }})
        """
        try:
            execute(query)
        except:
            print(query)

In [12]:
%%time
create_papers()

 34%|█████████████▎                         | 1673/4919 [00:25<00:47, 68.30it/s]


        CREATE (n:Paper {
            paper_id: "2703789238c80cc96e2a9d8dcc695d50316271ad",
            title: "Datenbanksysteme für Business, Technologie und Web (BTW 2017), 17. Fachtagung des GI-Fachbereichs „Datenbanken und Informationssysteme" (DBIS), 6.-10. März 2017, Stuttgart, Germany, Workshopband",
            
            year: toInteger(2017),
            fieldsOfStudy: ['Computer Science'],
            publicationDate: date("1970-01-01"),
            abstract: "The energy domain currently struggles with radical legal and technological changes, such as, smart meters. This results in new use cases which can be implemented based on business process technology. Understanding and automating business processes requires to model and test them. However, existing process testing approaches frequently struggle with the testing of process resources, such as ERP systems, and negative testing. Hence, this work presents a toolchain which tackles that limitations. The approach uses an op

100%|███████████████████████████████████████| 4919/4919 [01:15<00:00, 65.54it/s]

CPU times: user 45 s, sys: 1.24 s, total: 46.2 s
Wall time: 1min 15s


In [13]:
def create_paper__paper_id__range_index():
    """
    Create indexes
    """
    query = """
        CREATE RANGE INDEX paper__paper_id__range_index IF NOT EXISTS
        FOR (n:Paper)
        ON (n.paper_id)
    """

    execute(query)

In [14]:
%%time
create_paper__paper_id__range_index()

CPU times: user 1.91 ms, sys: 86 µs, total: 1.99 ms
Wall time: 5.6 ms


## Authors

In [15]:
def create_authors() -> None:
    """
    For each paper, generate a node with label `Author` for that paper.
    We are using the MERGE here since we don't want to duplicate authors.
    """
    for fname in tqdm(os.listdir(SEMANTIC_PATH)):
        # print(f'Creating the authors of {fname}')

        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())
    
        for author in paper['authors']:
            query = f"""
            MERGE (n:Author {{
                name: "{author['name']}",
                author_id: "{author['authorId']}"
            }})
            """
            execute(query)

In [16]:
%%time
create_authors()

100%|███████████████████████████████████████| 4935/4935 [01:26<00:00, 56.76it/s]

CPU times: user 21 s, sys: 3.56 s, total: 24.6 s
Wall time: 1min 26s


In [17]:
def create_author__author_id__range_index():
    """
    Create indexes
    """
    query = """
        CREATE RANGE INDEX author__author_id__range_index IF NOT EXISTS
        FOR (n:Author)
        ON (n.author_id)
    """

    execute(query)

In [18]:
%%time
create_author__author_id__range_index()

CPU times: user 2.96 ms, sys: 275 µs, total: 3.24 ms
Wall time: 5.43 ms


In [19]:
def link_author_to_paper() -> None:
    """
    Create the edge `Wrote` and `IsCorrespondingAuthor`, linking Authors and Papers.
    The first author is considered the corresponding author.
    """
    for fname in tqdm(os.listdir(SEMANTIC_PATH)):
        # print(f'Linking authors of file {fname}')

        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())

            is_first = True
            for author in paper['authors']:
                if is_first:
                    # The first author is the main corresponding author.
                    query = f"""
                        MATCH (a:Author {{author_id: '{author['authorId']}'}})
                        WITH a
                        MATCH (p:Paper {{paper_id: '{paper['paperId']}'}})
                        WITH a, p
                        CREATE (a)-[e:IsCorrespondingAuthor]->(p);
                    """
                    execute(query)
                    is_first = False
                
                query = f"""
                    MATCH (a:Author {{author_id: '{author['authorId']}'}})
                    WITH a
                    MATCH (p:Paper {{paper_id: '{paper['paperId']}'}})
                    WITH a, p
                    CREATE (a)-[e:Wrote]->(p);
                """
    
                execute(query)

In [20]:
%%time
link_author_to_paper()

100%|███████████████████████████████████████| 4962/4962 [03:18<00:00, 24.94it/s]

CPU times: user 22.9 s, sys: 4.56 s, total: 27.4 s
Wall time: 3min 18s


## Citations

In [21]:
def link_citations_between_papers() -> None:
    """
    Generate the edge Cited linking a Paper to a Paper.
    """
    for fname in tqdm(os.listdir(SEMANTIC_PATH)):
        # print(f'Linking citations of file {fname}')

        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())

        for citation in paper.get('citations', []):
            query = f"""
                MATCH (a:Paper {{paper_id: '{citation['paperId']}'}}), (p:Paper {{paper_id: '{paper['paperId']}'}})
                CREATE (a)-[e:Cites]->(p);
            """
            execute(query)

In [22]:
%%time
link_citations_between_papers()

100%|███████████████████████████████████████| 5050/5050 [05:09<00:00, 16.34it/s]

CPU times: user 45.6 s, sys: 7.96 s, total: 53.6 s
Wall time: 5min 9s


## Journals and Conferences

In [23]:
def create_journals() -> None:
    """
    Create the Journal nodes.
    """
    for fname in tqdm(os.listdir(SEMANTIC_PATH)):
        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())

        journal_name = parse_journal_name(paper)

        if journal_name != DEFAULT_JOURNAL_NAME:
            query = f"""
                MERGE (n:Journal {{
                    year: toInteger({paper['year']}),
                    name: "{journal_name}"
                    
                }})
            """

            execute(query)

In [24]:
%%time
create_journals()

100%|██████████████████████████████████████| 5192/5192 [00:25<00:00, 201.72it/s]

CPU times: user 5.44 s, sys: 1.11 s, total: 6.55 s
Wall time: 25.7 s


In [25]:
def link_journals()-> None:
    """
    Link a Paper to a Journal creating the `PublishedIn` edge.
    """
    for fname in tqdm(os.listdir(SEMANTIC_PATH)):
        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())

        query = f"""
            MATCH (p:Paper {{paper_id: '{paper['paperId']}'}})
                , (j:Journal {{name: '{parse_journal_name(paper)}', year: toInteger({paper['year'] if paper['year'] else -1})}})
            WITH p, j
            CREATE (p)-[e:PublishedIn]->(j);
        """
        execute(query)

In [26]:
%%time
link_journals()

100%|██████████████████████████████████████| 5206/5206 [00:42<00:00, 123.21it/s]

CPU times: user 4.9 s, sys: 853 ms, total: 5.75 s
Wall time: 42.3 s


In [27]:
def change_to_conference() -> None:
    """
    Change the label from Journal to Conference if the "Journal" name contains 'conference' in it.
    """
    query = """
        MATCH (j:Journal)
        WHERE toLower(j.name) =~ '.*conference.*' OR toLower(j.name) =~ '.*workshop.*'
        REMOVE j:Journal
        SET j:ConfWork
    """
    
    execute(query)

In [28]:
%%time
change_to_conference()

CPU times: user 2.58 ms, sys: 365 µs, total: 2.94 ms
Wall time: 29.5 ms


## Reviews

We will have to generate synthetic data here to represent the reviews.

Typically, each paper has 3 reviewers, who are usually relevant authors.
The author cannot review its own paper.

The strategy that we will be using is to select up to 3 authors who:
1. wrote papers cited by the paper in question; and
2. didn't wrote the paper itself.

In [29]:
def get_possible_reviewers():
    """
    Auxiliary function that returns an aggregation of all possible reviewers of a paper.
    The logic of a "possible reviewer" is to select an author who:
    1. wrote paper(s) cited by the paper in question; and who
    2. didn't wrote the paper itself.
    """
    query = """
        MATCH (a:Author)-[w1:Wrote]->(mp:Paper)-[c:Cites]->(cp:Paper)
        WITH mp, cp, a
        MATCH (wcp:Author)-[w2:Wrote]->(cp)
        WHERE NOT (wcp)-[:Wrote]->(mp)
        RETURN mp.paper_id AS paper_id, collect(wcp.author_id) AS possible_reviewer_ids;
    """

    return execute(query)

In [30]:
def link_reviewer_to_paper() -> None:
    """
    This function generates synthetic data.
    """
    result = get_possible_reviewers()
    
    for paper_id, possible_reviewers in tqdm(result[0]):
        # Papers can have a different amount of reviewers, varying from 1 to 4, following the distribution specified by `p`.
        # Edge case: If the paper doesn't cite any other paper, it will have 0 reviewers.
        reviewer_qty = min(
            np.random.choice(np.arange(1, 5), p=[0.1, 0.3, 0.5, 0.1]),
            len(possible_reviewers)
        )

        reviewers = random.sample(possible_reviewers, reviewer_qty)
        for reviewer in reviewers:
            query = f"""
                MATCH (a:Author {{author_id: '{reviewer}'}}), (p:Paper {{paper_id: '{paper_id}'}})
                CREATE (a)-[e:Reviewed]->(p);
            """

            execute(query)

In [31]:
%%time
link_reviewer_to_paper()

100%|███████████████████████████████████████| 4880/4880 [00:52<00:00, 92.18it/s]

CPU times: user 8.52 s, sys: 1.23 s, total: 9.75 s
Wall time: 53.4 s


# Queries

<b>Query 1</b>

Find the top 3 most cited papers of each conference.

```
MATCH (p:Paper)-[:cited]->(cited:Paper) WITH p.name AS journal, p.title AS title, COUNT(*) AS num_citations ORDER BY journal, num_citations DESC WITH journal, COLLECT({title: title, num_citations: num_citations}) AS papers WITH journal, papers, [i IN RANGE(1, SIZE(papers)) | i] AS ranks UNWIND ranks AS rank WITH journal, papers[rank - 1].title AS title, papers[rank - 1].num_citations AS num_citations, rank WHERE rank <= 3 RETURN journal, title, num_citations, rank ORDER BY journal, rank
```

<b>Manually Test</b>

```
MATCH (p:Paper)-[c:cited]->(cited:Paper) WITH p.name AS journal, p.title AS title, COUNT(c) AS num_citations WHERE journal = '2014 IEEE International Conference on Big Data (Big Data)' RETURN journal, title, num_citations ORDER BY num_citations DESC
```

<b>Query 3</b>

Find the impact factors of the journals in your graph (see https://en.wikipedia.org/wiki/Impact_factor, for the definition of the impact factor).

```
MATCH (citing_paper:Paper)-[:Cites]->(published_paper:Paper {year: j.year})-[:PublishedIn]->(j:Journal)
WITH COUNT(DISTINCT citing_paper) AS total_citations, j.name AS journal_name, j AS j1
MATCH (j2: Journal)<-[:PublishedIn]-(p:Paper)
WHERE j2.year IN [j1.year - 1, j1.year - 2]
      AND j1.name = j2.name
WITH j1.year AS year,
     COUNT(p.title) AS past_publications,
     j1.name AS journal_name,
     total_citations
RETURN year, journal_name, total_citations, past_publications, 1.0 * total_citations / past_publications
ORDER BY journal_name, year;
```

<b>Query 4</b>

Find the h-indexes of the authors in your graph

```
MATCH (a:Author)-[:Wrote]->(p:Paper)-[:cited]->(cited:Paper) WITH a, p, COUNT(*) AS num_citations ORDER BY num_citations DESC WITH a, COLLECT(num_citations) AS citation_counts WITH a, [i IN RANGE(1, SIZE(citation_counts)) | CASE WHEN citation_counts[i - 1] >= i THEN i ELSE 0 END] AS h_values WITH a, MAX(h_values) AS h_index WITH a, MAX(REDUCE(s = 0, h IN h_index | CASE WHEN h > s THEN h ELSE s END)) AS max_h_index RETURN a.author_id AS author_id, a.name AS author_name, max_h_index
```

<b>Manually test</b>

```
MATCH (a:Author)-[:Wrote]->(p:Paper)-[:cited]->(cited:Paper) WITH a, p, COUNT(*) AS num_citations ORDER BY num_citations DESC WHERE a.name = 'M. Mokbel' return a.name, p.title, num_citations
```

In [32]:
MATCH (a:Author)-[:Wrote]->(p:Paper)-[:PublishedIn]->(c:Journal)
// WITH a, c, size(collect(c.year)) AS s
// WHERE s > 2
// RETURN a.name, collect(c.year), c.name
// ORDER BY a.name, c.name
WITH a.name AS author, collect(DISTINCT c.year) AS years, c.name AS conference
WHERE size(years) > 1
RETURN author, years, conference
// ORDER BY  DESC
// RETURN a.name, size(collect(c.name)), collect(c.name)
// ORDER BY size(collect(c.name)) DESC

SyntaxError: invalid syntax (3593147951.py, line 1)

# Recommender System

<b>Part 1</b>

```
// First we are looking for papers containing any of those keywords.
MATCH (p:Paper)
WHERE
    // Could've been an array intersection, but APOC was giving us some setup issues.
    'data management' IN p.keywords
    OR 'indexing' IN p.keywords
    OR 'data modeling' IN p.keywords
    OR 'big data' IN p.keywords
    OR 'data processing' IN p.keywords
    OR 'data storage' IN p.keywords
    OR 'data querying' IN p.keywords
RETURN *
```

<b>Part 2</b>

```
// Now we want the conferences or journals with at least 90% of published papers being related to databases.
MATCH (p:Paper)-[:PublishedIn]->(jc)
WITH p, (
        'data management' IN p.keywords
        OR 'indexing' IN p.keywords
        OR 'data modeling' IN p.keywords
        OR 'big data' IN p.keywords
        OR 'data processing' IN p.keywords
        OR 'data storage' IN p.keywords
        OR 'data querying' IN p.keywords
    ) AS in_db_community,
    jc
WITH COUNT(p) AS total_published_papers, SUM(CASE in_db_community WHEN TRUE THEN 1 ELSE 0 END) AS db_comm_papers, jc.name AS jc_name
WHERE 100.0 * db_comm_papers / total_published_papers > 90.0 
RETURN total_published_papers, db_comm_papers, 100.0 * db_comm_papers / total_published_papers AS percentage_of_db_papers, jc_name
LIMIT 50
```

<b>Part 3</b>

```
// Let's now grab the top 100 most cited papers in the Database community.
MATCH (p:Paper)-[:PublishedIn]->(jc)
WITH p, (
        'data management' IN p.keywords
        OR 'indexing' IN p.keywords
        OR 'data modeling' IN p.keywords
        OR 'big data' IN p.keywords
        OR 'data processing' IN p.keywords
        OR 'data storage' IN p.keywords
        OR 'data querying' IN p.keywords
    ) AS in_db_community,
    jc
WITH COUNT(p) AS total_published_papers, SUM(CASE in_db_community WHEN TRUE THEN 1 ELSE 0 END) AS db_comm_papers, jc.name AS jc_name, jc
WHERE 100.0 * db_comm_papers / total_published_papers > 90.0
WITH collect(jc.name) AS db_comm_conferences

MATCH (citing_paper:Paper)-[:Cites]->(cited_paper:Paper)-[:PublishedIn]->(jc1), (citing_paper)-[:PublishedIn]->(jc2)
WHERE jc1.name IN db_comm_conferences
  AND jc2.name IN db_comm_conferences
WITH cited_paper, jc1, COUNT(DISTINCT citing_paper) AS c
RETURN c, jc1.name, cited_paper.title
ORDER BY c DESC
LIMIT 100
```

<b>Part 4</b>

```
// Now, we will find the gurus of the community.
MATCH (p:Paper)-[:PublishedIn]->(jc)
WITH p, (
        'data management' IN p.keywords
        OR 'indexing' IN p.keywords
        OR 'data modeling' IN p.keywords
        OR 'big data' IN p.keywords
        OR 'data processing' IN p.keywords
        OR 'data storage' IN p.keywords
        OR 'data querying' IN p.keywords
    ) AS in_db_community,
    jc
WITH COUNT(p) AS total_published_papers, SUM(CASE in_db_community WHEN TRUE THEN 1 ELSE 0 END) AS db_comm_papers, jc.name AS jc_name, jc
WHERE 100.0 * db_comm_papers / total_published_papers > 90.0
WITH collect(jc.name) AS db_comm_conferences

MATCH (citing_paper:Paper)-[:Cites]->(cited_paper:Paper)-[:PublishedIn]->(jc1), (citing_paper)-[:PublishedIn]->(jc2)
WHERE jc1.name IN db_comm_conferences
  AND jc2.name IN db_comm_conferences
WITH cited_paper, jc1, COUNT(DISTINCT citing_paper) AS c
WITH COLLECT(cited_paper.paper_id)[1..100] AS most_cited_papers// UNWIND most_cited_papers AS most_cited_paper

MATCH (p1:Paper)<-[:Wrote]-(a:Author)-[:Wrote]->(p2:Paper)
WHERE p1 <> p2
AND p1.paper_id IN most_cited_papers
AND p2.paper_id IN most_cited_papers
RETURN a

// , collect() AS col UNWIND RANGE(0, SIZE(col) - 1) AS un
// RETURN un AS rn, col[un][0] AS cited_paper_node//, col[un][1] AS jc_node

// WITH , AS rank


// MATCH (a:Author)-[:Wrote]->(cited_paper)

// RETURN c, jc1.name, cited_paper.title
// ORDER BY c ASC
LIMIT 100
```

# Known limitations

1. Our data source doesn't always provide the Journal name.
When we don't have it, we are not creating a node for the journal where that paper was published.
2. Papers without abstracts won't have their keywords extracted.

In [ ]:
MATCH (citing_paper:Paper)-[:Cites]->(published_paper:Paper {year: j.year})-[:PublishedIn]->(j:Journal)
WITH COUNT(DISTINCT citing_paper) AS total_citations, j.name AS journal_name, j AS j1
MATCH (j2: Journal)<-[:PublishedIn]-(p:Paper)
WHERE j2.year IN [j1.year - 1, j1.year - 2]
      AND j1.name = j2.name
WITH j1.year AS year,
     COUNT(p.title) AS past_publications,
     j1.name AS journal_name,
     total_citations
RETURN year, journal_name, total_citations, past_publications, 1.0 * total_citations / past_publications
ORDER BY journal_name, year
LIMIT 50;